In [1]:
import os
import cv2
import numpy as np

In [2]:
participant_id = "AP_05"
path = os.path.join("data", participant_id, "thermal-raw")
#path = os.path.join("/Volumes/SANDISK USB", "data", participant_id, "thermal-raw")

frame_count = 0
all_frames = []
frame_thresholds = []  # Store threshold for each frame

left_right = ['Frag_13.avi', 'Frag_19.avi']
left_right_frames = []

count_left_right = 0 
count_supine = 0

threshold = 0

for file_name in os.listdir(path):
    # Skip files starting with ._
    if file_name.startswith("._"):
        continue
    
    if file_name.endswith(".avi") and not file_name.startswith("Frag_1.avi"):
        print(f"Processing video: {file_name}, Frames processed: {frame_count}")

        # Determine the frame interval and threshold for this video
        if file_name in left_right:
            # For left/right position videos
            frame_interval = 2000  # Capture every 2 seconds
            threshold = 5  # Lower threshold for frame similarity
        else:
            frame_interval = 5000  # Capture every 5 seconds
            threshold = 16  # Default threshold

        # Open the video file
        video_path = os.path.join(path, file_name)
        video = cv2.VideoCapture(video_path)
        
        if not video.isOpened():
            print(f"Error: Could not open video file {file_name}")
            continue

        while True:
            ret, frame = video.read()
            if not ret:
                video.release()  
                cv2.destroyAllWindows()  
                break

            if not video.get(cv2.CAP_PROP_FRAME_COUNT):
                print(f"Warning: First frame of {file_name} is not a keyframe")
                
            video.set(cv2.CAP_PROP_POS_MSEC, video.get(cv2.CAP_PROP_POS_MSEC) + frame_interval)

            all_frames.append(frame)
            frame_thresholds.append(threshold)  # Store the threshold value
            frame_count += 1

            if file_name in left_right:
                left_right_frames.append(frame)

            cv2.imshow("Frame", frame) 
            if frame_interval == 2000:
                count_left_right += 1
            elif frame_interval == 5000:
                count_supine += 1

            if cv2.waitKey(1) & 0xFF == ord('q'):
                video.release()
                cv2.destroyAllWindows()
                break
print("\nVideo processed")


Processing video: Frag_19.avi, Frames processed: 0
Processing video: Frag_31.avi, Frames processed: 121
Processing video: Frag_25.avi, Frames processed: 242
Processing video: Frag_4.avi, Frames processed: 363
Processing video: Frag_5.avi, Frames processed: 484
Processing video: Frag_24.avi, Frames processed: 605
Processing video: Frag_30.avi, Frames processed: 726
Processing video: Frag_18.avi, Frames processed: 847
Processing video: Frag_26.avi, Frames processed: 968
Processing video: Frag_32.avi, Frames processed: 1089
Processing video: Frag_7.avi, Frames processed: 1210
Processing video: Frag_6.avi, Frames processed: 1331
Processing video: Frag_33.avi, Frames processed: 1452
Processing video: Frag_27.avi, Frames processed: 1573
Processing video: Frag_23.avi, Frames processed: 1694
Processing video: Frag_2.avi, Frames processed: 1815
Processing video: Frag_3.avi, Frames processed: 1936
Processing video: Frag_36.avi, Frames processed: 2057
Processing video: Frag_22.avi, Frames process

In [3]:
all_frames = np.array(all_frames) # Convert the list of frames to a NumPy array 
print(f"Total frames processed: {frame_count}")

# Print the number of frames captured using different intervals
print(f"\nNumber of frames captured with 2000 ms interval: {count_left_right}")
print(f"Number of frames captured with 5000 ms interval: {count_supine}")

Total frames processed: 4330

Number of frames captured with 2000 ms interval: 0
Number of frames captured with 5000 ms interval: 4330


In [4]:
print("Total frames extracted:", all_frames.shape[0]) 

unique_frames = []
unique_left_right_count = 0

# Loop through each frame
for i in range(all_frames.shape[0]):
    is_unique = True
    current_threshold = frame_thresholds[i]

    for frame in unique_frames:
        diff = cv2.absdiff(all_frames[i], frame)
        if np.mean(diff) < current_threshold:
            is_unique = False
            break

    if is_unique:
        unique_frames.append(all_frames[i])

unique_frames = np.array(unique_frames)
left_right_frames = np.array(left_right_frames)


for unique_frame in unique_frames:
    for lr_frame in left_right_frames:
        if np.array_equal(unique_frame, lr_frame):
            unique_left_right_count += 1
            break

print("Number of unique frames:", unique_frames.shape[0])
print(f"Number of unique frames from left/right videos: {unique_left_right_count}")
print(f"Percentage of unique frames from left/right videos: {(unique_left_right_count / unique_frames.shape[0]) * 100:.2f}%")

Total frames extracted: 4330
Number of unique frames: 180
Number of unique frames from left/right videos: 0
Percentage of unique frames from left/right videos: 0.00%


In [6]:
# save the unique frames
frame_dir = os.path.join("data", participant_id, "frames_new")
#frame_dir = os.path.join("/Volumes/SANDISK USB", "data", participant_id, "frames")
if not os.path.exists(frame_dir):
    os.makedirs(frame_dir)

# Save each unique frame as an image
for i in range(unique_frames.shape[0]):
    image_path = os.path.join(frame_dir, participant_id + "_" + str(i) + ".jpg")
    cv2.imwrite(image_path, unique_frames[i])
print("Unique frames saved to disk for participant: " + participant_id)

Unique frames saved to disk for participant: AP_05


: 

In [ ]:
# Display all frames
# for img in all_frames:
#     cv2.imshow("All Frames", img)
#     if cv2.waitKey(1) & 0xFF == ord('q'):  
#         cv2.destroyAllWindows()
#         break
# cv2.destroyAllWindows()

In [ ]:
# Display unique frames
# for img in unique_frames:
#     cv2.imshow("Unique Frames", img)
#     if cv2.waitKey(10) & 0xFF == ord('q'):  
#         cv2.destroyAllWindows()
#         break
# cv2.destroyAllWindows()